In [76]:
# for interpreting eCAVIAR results

import glob
import pandas as pd

### mendelian list

In [17]:
filename = "2021.02.10.mend.genes.tsv"

mendelian_list = pd.read_csv(filename, sep = ' ', header = None)
mendelian_list.columns = ['Tissue', 'gene_name']

In [8]:
ensg_filename = "/net/home/dlee/dataset/ENSP_ENST_ENSG_names.csv"
ensg_df = pd.read_csv(ensg_filename)

In [23]:
def get_ensg_name(gene_name, ensg_df):
    if len(ensg_df.loc[ensg_df["Gene name"] == gene_name]) == 0:
        return None
    
    list1 = list(set(ensg_df.loc[ensg_df["Gene name"] == gene_name]["Gene stable ID"]))
    
    str1 = '|'.join(list1)
    return str1

In [25]:
mendelian_list["ENSG"] = [get_ensg_name(x, ensg_df) for x in mendelian_list["gene_name"]]

In [27]:
mendelian_list.loc[mendelian_list["ENSG"] == None]

,Tissue,gene_name,ENSG


In [28]:
mendelian_list.head()

,Tissue,gene_name,ENSG
0,LDL,PCSK9,ENSG00000169174
1,LDL,APOB,ENSG00000084674
2,LDL,APOE,ENSG00000130203
3,LDL,APOC2,ENSG00000234906
4,LDL,LPL,ENSG00000175445


In [54]:
def get_ecaviar_results(mendelian, ecaviar_directory, coloc_cutoff):
    efiles = glob.glob(ecaviar_directory + '*col')
    print("number of ecaviar_files: ", str(len(efiles)))
    
    #organize file into a dataframe
    efiles_nodir = [i.split("/")[-1] for i in efiles]
    position = [int(i.split("_")[1]) for i in efiles_nodir]
    genes = [i.split("_")[2] for i in efiles_nodir]

    data = {'Filename': efiles, 'Locus': position, 'Gene': genes}
    efile_df = pd.DataFrame(data)

    efile_df = efile_df.sort_values(['Locus'])
    
    #check what mendelian files are in the list of hits
    output_list = []

    #print number of unique locus
    print("number of unique locus ", len(efile_df["Locus"].unique()))

    counter = 0

    for i in efile_df["Locus"].unique():

        efile_locus_df = efile_df[efile_df["Locus"] == i]

        output_string = str(i) + ': '

        output_string = output_string + str(len(efile_locus_df)) + ": "

        for index, row in efile_locus_df.iterrows():
            eCAVIAR_output = pd.read_csv(row["Filename"] , sep= '\t')
            maxCLPP = max(eCAVIAR_output["CLPP"])
            maxPCausalset = max(eCAVIAR_output["Prob_in_pCausalSet"])

            if maxCLPP > coloc_cutoff:
                output_string = output_string + str(row["Gene"]) + ", "

        counter += 1
        
#        locus: 55519174: 21: ENSG00000169174.9, 


        output_list.append(output_string)

    genes = [x.split(':')[-1][1:-3] for x in output_list]
    colocalized_gene_set = []
    for i in genes:
        if len(i) > 1:
            for name in i.split(", "):
                colocalized_gene_set.append(name)

    colocalized_gene_set = [i.split(".")[0] for i in colocalized_gene_set]

    total_gene_set = list(efile_df["Gene"].unique())
    total_gene_set = set([i.split(".")[0] for i in total_gene_set])

    total_mendelian = 0

    for i in total_gene_set:
        if i in mendelian:
            total_mendelian += 1


    colocalized_mendelian = 0

    for i in colocalized_gene_set:
        if i in mendelian:
            print(i)
            colocalized_mendelian += 1    
            
    for i in output_list:
        for j in mendelian:
            if j == None:
                continue
            if "|" in j:
                for name in j.split("|"):
                    if name in i:                    
                        print(i)
                        print(len(i.split(","))-1)
                    
            elif j in i:
                print(i)
                print(len(i.split(","))-1)
                
    print("colocalized mendelian: " , colocalized_mendelian)
    print("colocalized non-mendel: ", len(colocalized_gene_set) - colocalized_mendelian)
    print("colocalized total: ", len(colocalized_gene_set))

    print("non-colocalized mendelian: " , total_mendelian - colocalized_mendelian)
    print("non-mendel non-coloc: ", len(total_gene_set) - len(colocalized_gene_set) - (total_mendelian - colocalized_mendelian))
    print("total non-coloc: ", len(total_gene_set) - len(colocalized_gene_set))

    print("total mendelian: " , total_mendelian)     
    print("total non-mendel: ", len(total_gene_set) - total_mendelian)
    print("total     set: ", len(total_gene_set))

### test for Breast Cancer

In [55]:
mendelian = list(mendelian_list[mendelian_list["Tissue"] == "BC"]["ENSG"])


In [57]:
mendelian_new = []

for ensg in mendelian:
    if "|" in ensg:
        mendelian_new.extend(ensg.split("|"))
    else:
        mendelian_new.append(ensg)

In [60]:
len(mendelian_new)

39

In [59]:
ecaviar_directory = '/net/home/dlee/GTEx/gwas/zhang_2020_bc/eCAVIAR_results/eCAVIAR_output/Breast_Mammary_Tissue/'

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.5)

number of ecaviar_files:  17846
number of unique locus  561
colocalized mendelian:  0
colocalized non-mendel:  18
colocalized total:  18
non-colocalized mendelian:  17
non-mendel non-coloc:  9619
total non-coloc:  9636
total mendelian:  17
total non-mendel:  9637
total     set:  9654


### test for t2d

In [62]:
mendelian = list(mendelian_list[mendelian_list["Tissue"] == "T2D"]["ENSG"])

mendelian_new = []

for ensg in mendelian:
    if ensg == None:
        continue        
    if "|" in ensg:
        mendelian_new.extend(ensg.split("|"))
    else:
        mendelian_new.append(ensg)

In [64]:
ecaviar_directory = '/net/data/GTEx/gwas/mahajan_2018_t2d/eCAVIAR_results/eCAVIAR_output/Liver/'

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.5)

number of ecaviar_files:  6280
number of unique locus  251
colocalized mendelian:  0
colocalized non-mendel:  1
colocalized total:  1
non-colocalized mendelian:  9
non-mendel non-coloc:  4149
total non-coloc:  4158
total mendelian:  9
total non-mendel:  4150
total     set:  4159


In [65]:
get_ecaviar_results(mendelian_new, ecaviar_directory, 0.2)

number of ecaviar_files:  6280
number of unique locus  251
colocalized mendelian:  0
colocalized non-mendel:  8
colocalized total:  8
non-colocalized mendelian:  9
non-mendel non-coloc:  4142
total non-coloc:  4151
total mendelian:  9
total non-mendel:  4150
total     set:  4159


In [66]:
ecaviar_directory = '/net/data/GTEx/gwas/mahajan_2018_t2d/eCAVIAR_results/eCAVIAR_output/Pancreas/'
get_ecaviar_results(mendelian_new, ecaviar_directory, 0.5)

number of ecaviar_files:  6345
number of unique locus  251
colocalized mendelian:  0
colocalized non-mendel:  284
colocalized total:  284
non-colocalized mendelian:  10
non-mendel non-coloc:  3902
total non-coloc:  3912
total mendelian:  10
total non-mendel:  4186
total     set:  4196


In [67]:
get_ecaviar_results(mendelian_new, ecaviar_directory, 0.2)

number of ecaviar_files:  6345
number of unique locus  251
colocalized mendelian:  0
colocalized non-mendel:  298
colocalized total:  298
non-colocalized mendelian:  10
non-mendel non-coloc:  3888
total non-coloc:  3898
total mendelian:  10
total non-mendel:  4186
total     set:  4196


In [68]:
ecaviar_directory = '/net/data/GTEx/gwas/mahajan_2018_t2d/eCAVIAR_results/eCAVIAR_output/Whole_Blood/'

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.5)

number of ecaviar_files:  5787
number of unique locus  250
colocalized mendelian:  0
colocalized non-mendel:  255
colocalized total:  255
non-colocalized mendelian:  5
non-mendel non-coloc:  3602
total non-coloc:  3607
total mendelian:  5
total non-mendel:  3857
total     set:  3862


In [69]:
get_ecaviar_results(mendelian_new, ecaviar_directory, 0.2)

number of ecaviar_files:  5787
number of unique locus  250
colocalized mendelian:  0
colocalized non-mendel:  274
colocalized total:  274
non-colocalized mendelian:  5
non-mendel non-coloc:  3583
total non-coloc:  3588
total mendelian:  5
total non-mendel:  3857
total     set:  3862


### test for UC

In [70]:
mendelian = list(mendelian_list[mendelian_list["Tissue"] == "UC"]["ENSG"])

mendelian_new = []

for ensg in mendelian:
    if ensg == None:
        continue        
    if "|" in ensg:
        mendelian_new.extend(ensg.split("|"))
    else:
        mendelian_new.append(ensg)

In [74]:
ecaviar_directory = '/net/data/GTEx/gwas/liu_2015_and_goyette_2015_ibd/uc_eCAVIAR/eCAVIAR_output/Colon_Transverse/'

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.5)

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.2)

number of ecaviar_files:  14138
number of unique locus  371
colocalized mendelian:  0
colocalized non-mendel:  677
colocalized total:  677
non-colocalized mendelian:  5
non-mendel non-coloc:  7779
total non-coloc:  7784
total mendelian:  5
total non-mendel:  8456
total     set:  8461
number of ecaviar_files:  14138
number of unique locus  371
colocalized mendelian:  0
colocalized non-mendel:  730
colocalized total:  730
non-colocalized mendelian:  5
non-mendel non-coloc:  7726
total non-coloc:  7731
total mendelian:  5
total non-mendel:  8456
total     set:  8461


In [72]:
ecaviar_directory = '/net/data/GTEx/gwas/liu_2015_and_goyette_2015_ibd/uc_eCAVIAR/eCAVIAR_output/Colon_Sigmoid/'
get_ecaviar_results(mendelian_new, ecaviar_directory, 0.5)

number of ecaviar_files:  13713
number of unique locus  371
colocalized mendelian:  0
colocalized non-mendel:  654
colocalized total:  654
non-colocalized mendelian:  4
non-mendel non-coloc:  7546
total non-coloc:  7550
total mendelian:  4
total non-mendel:  8200
total     set:  8204


In [73]:
get_ecaviar_results(mendelian_new, ecaviar_directory, 0.2)

number of ecaviar_files:  13713
number of unique locus  371
colocalized mendelian:  0
colocalized non-mendel:  703
colocalized total:  703
non-colocalized mendelian:  4
non-mendel non-coloc:  7497
total non-coloc:  7501
total mendelian:  4
total non-mendel:  8200
total     set:  8204


In [75]:
ecaviar_directory = '/net/data/GTEx/gwas/liu_2015_and_goyette_2015_ibd/uc_eCAVIAR/eCAVIAR_output/Small_Intestine_Terminal_Ileum/'

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.5)

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.2)

number of ecaviar_files:  14478
number of unique locus  371
colocalized mendelian:  0
colocalized non-mendel:  694
colocalized total:  694
non-colocalized mendelian:  5
non-mendel non-coloc:  8019
total non-coloc:  8024
total mendelian:  5
total non-mendel:  8713
total     set:  8718
number of ecaviar_files:  14478
number of unique locus  371
colocalized mendelian:  0
colocalized non-mendel:  729
colocalized total:  729
non-colocalized mendelian:  5
non-mendel non-coloc:  7984
total non-coloc:  7989
total mendelian:  5
total non-mendel:  8713
total     set:  8718


### test for CD

In [79]:
mendelian = list(mendelian_list[mendelian_list["Tissue"] == "CD"]["ENSG"])

mendelian_new = []

for ensg in mendelian:
    if ensg == None:
        continue        
    if "|" in ensg:
        mendelian_new.extend(ensg.split("|"))
    else:
        mendelian_new.append(ensg)

In [81]:
ecaviar_directory = '/net/data/GTEx/gwas/liu_2015_and_goyette_2015_ibd/cd_eCAVIAR/eCAVIAR_output/Colon_Sigmoid/'

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.5)

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.2)

number of ecaviar_files:  13798
number of unique locus  371
ENSG00000167207
50304249: 19: ENSG00000166164.11, ENSG00000260249.2, ENSG00000205423.7, ENSG00000155393.8, ENSG00000261685.2, ENSG00000102935.7, ENSG00000121281.8, ENSG00000140807.4, ENSG00000205414.1, ENSG00000103449.7, ENSG00000260769.1, ENSG00000261644.1, ENSG00000167208.10, ENSG00000083799.13, ENSG00000260929.1, ENSG00000167207.7, ENSG00000121274.8, 
17
colocalized mendelian:  1
colocalized non-mendel:  903
colocalized total:  904
non-colocalized mendelian:  8
non-mendel non-coloc:  7011
total non-coloc:  7019
total mendelian:  9
total non-mendel:  7914
total     set:  7923
number of ecaviar_files:  13798
number of unique locus  371
ENSG00000167207
50304249: 19: ENSG00000166164.11, ENSG00000260249.2, ENSG00000205423.7, ENSG00000155393.8, ENSG00000261685.2, ENSG00000102935.7, ENSG00000121281.8, ENSG00000140807.4, ENSG00000205414.1, ENSG00000103449.7, ENSG00000260769.1, ENSG00000261644.1, ENSG00000167208.10, ENSG00000083799.

In [82]:
ecaviar_directory = '/net/data/GTEx/gwas/liu_2015_and_goyette_2015_ibd/cd_eCAVIAR/eCAVIAR_output/Small_Intestine_Terminal_Ileum/'

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.5)

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.2)

number of ecaviar_files:  14695
number of unique locus  371
ENSG00000167207
50304249: 21: ENSG00000167208.10, ENSG00000261685.2, ENSG00000121274.8, ENSG00000260381.2, ENSG00000140807.4, ENSG00000270120.1, ENSG00000260769.1, ENSG00000102935.7, ENSG00000166164.11, ENSG00000205423.7, ENSG00000083799.13, ENSG00000155393.8, ENSG00000260249.2, ENSG00000103449.7, ENSG00000261644.1, ENSG00000167207.7, ENSG00000205414.1, ENSG00000121281.8, ENSG00000260929.1, 
19
colocalized mendelian:  1
colocalized non-mendel:  929
colocalized total:  930
non-colocalized mendelian:  9
non-mendel non-coloc:  7581
total non-coloc:  7590
total mendelian:  10
total non-mendel:  8510
total     set:  8520
number of ecaviar_files:  14695
number of unique locus  371
ENSG00000167207
50304249: 21: ENSG00000167208.10, ENSG00000261685.2, ENSG00000121274.8, ENSG00000260381.2, ENSG00000140807.4, ENSG00000270120.1, ENSG00000260769.1, ENSG00000102935.7, ENSG00000166164.11, ENSG00000205423.7, ENSG00000083799.13, ENSG0000015539

In [83]:
ecaviar_directory = '/net/data/GTEx/gwas/liu_2015_and_goyette_2015_ibd/cd_eCAVIAR/eCAVIAR_output/Colon_Transverse/'

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.5)

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.2)

number of ecaviar_files:  14287
number of unique locus  371
colocalized mendelian:  0
colocalized non-mendel:  13
colocalized total:  13
non-colocalized mendelian:  10
non-mendel non-coloc:  8210
total non-coloc:  8220
total mendelian:  10
total non-mendel:  8223
total     set:  8233
number of ecaviar_files:  14287
number of unique locus  371
colocalized mendelian:  0
colocalized non-mendel:  36
colocalized total:  36
non-colocalized mendelian:  10
non-mendel non-coloc:  8187
total non-coloc:  8197
total mendelian:  10
total non-mendel:  8223
total     set:  8233


### test for HDL

In [84]:
mendelian = list(mendelian_list[mendelian_list["Tissue"] == "HDL"]["ENSG"])

mendelian_new = []

for ensg in mendelian:
    if ensg == None:
        continue        
    if "|" in ensg:
        mendelian_new.extend(ensg.split("|"))
    else:
        mendelian_new.append(ensg)

In [86]:
ecaviar_directory = '/net/data/GTEx/gwas/ukbb_hdl_conditioned/eCAVIAR/eCAVIAR_output/Whole_Blood/'

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.5)
print("")

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.2)

number of ecaviar_files:  7252
number of unique locus  245
colocalized mendelian:  0
colocalized non-mendel:  16
colocalized total:  16
non-colocalized mendelian:  6
non-mendel non-coloc:  3987
total non-coloc:  3993
total mendelian:  6
total non-mendel:  4003
total     set:  4009

number of ecaviar_files:  7252
number of unique locus  245
colocalized mendelian:  0
colocalized non-mendel:  19
colocalized total:  19
non-colocalized mendelian:  6
non-mendel non-coloc:  3984
total non-coloc:  3990
total mendelian:  6
total non-mendel:  4003
total     set:  4009


In [87]:
ecaviar_directory = '/net/data/GTEx/gwas/ukbb_hdl_conditioned/eCAVIAR/eCAVIAR_output/Liver/'

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.5)
print("")

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.2)

number of ecaviar_files:  7841
number of unique locus  249
colocalized mendelian:  0
colocalized non-mendel:  21
colocalized total:  21
non-colocalized mendelian:  7
non-mendel non-coloc:  4276
total non-coloc:  4283
total mendelian:  7
total non-mendel:  4297
total     set:  4304

number of ecaviar_files:  7841
number of unique locus  249
colocalized mendelian:  0
colocalized non-mendel:  26
colocalized total:  26
non-colocalized mendelian:  7
non-mendel non-coloc:  4271
total non-coloc:  4278
total mendelian:  7
total non-mendel:  4297
total     set:  4304


In [88]:
ecaviar_directory = '/net/data/GTEx/gwas/ukbb_hdl_conditioned/eCAVIAR/eCAVIAR_output/Adipose_Subcutaneous/'

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.5)
print("")

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.2)

number of ecaviar_files:  8221
number of unique locus  249
colocalized mendelian:  0
colocalized non-mendel:  28
colocalized total:  28
non-colocalized mendelian:  7
non-mendel non-coloc:  4521
total non-coloc:  4528
total mendelian:  7
total non-mendel:  4549
total     set:  4556

number of ecaviar_files:  8221
number of unique locus  249
ENSG00000100979
44545460: 48: ENSG00000271984.1, ENSG00000100979.10, 
2
colocalized mendelian:  1
colocalized non-mendel:  41
colocalized total:  42
non-colocalized mendelian:  6
non-mendel non-coloc:  4508
total non-coloc:  4514
total mendelian:  7
total non-mendel:  4549
total     set:  4556


### test for LDL

In [89]:
mendelian = list(mendelian_list[mendelian_list["Tissue"] == "LDL"]["ENSG"])

mendelian_new = []

for ensg in mendelian:
    if ensg == None:
        continue        
    if "|" in ensg:
        mendelian_new.extend(ensg.split("|"))
    else:
        mendelian_new.append(ensg)

In [92]:
ecaviar_directory = '/net/data/GTEx/gwas/ukbb_ldl_conditioned/eCAVIAR/eCAVIAR_output/Whole_Blood/'

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.5)
print("")

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.2)

number of ecaviar_files:  3469
number of unique locus  126
colocalized mendelian:  0
colocalized non-mendel:  13
colocalized total:  13
non-colocalized mendelian:  3
non-mendel non-coloc:  2085
total non-coloc:  2088
total mendelian:  3
total non-mendel:  2098
total     set:  2101

number of ecaviar_files:  3469
number of unique locus  126
ENSG00000169174
55519174: 21: ENSG00000169174.9, 
1
colocalized mendelian:  1
colocalized non-mendel:  15
colocalized total:  16
non-colocalized mendelian:  2
non-mendel non-coloc:  2083
total non-coloc:  2085
total mendelian:  3
total non-mendel:  2098
total     set:  2101


In [93]:
ecaviar_directory = '/net/data/GTEx/gwas/ukbb_ldl_conditioned/eCAVIAR/eCAVIAR_output/Liver/'

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.5)
print("")

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.2)

number of ecaviar_files:  3698
number of unique locus  126
colocalized mendelian:  0
colocalized non-mendel:  13
colocalized total:  13
non-colocalized mendelian:  5
non-mendel non-coloc:  2204
total non-coloc:  2209
total mendelian:  5
total non-mendel:  2217
total     set:  2222

number of ecaviar_files:  3698
number of unique locus  126
colocalized mendelian:  0
colocalized non-mendel:  14
colocalized total:  14
non-colocalized mendelian:  5
non-mendel non-coloc:  2203
total non-coloc:  2208
total mendelian:  5
total non-mendel:  2217
total     set:  2222


In [91]:
ecaviar_directory = '/net/data/GTEx/gwas/ukbb_ldl_conditioned/eCAVIAR/eCAVIAR_output/Adipose_Subcutaneous/'

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.5)
print("")

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.2)

number of ecaviar_files:  3834
number of unique locus  126
colocalized mendelian:  0
colocalized non-mendel:  11
colocalized total:  11
non-colocalized mendelian:  4
non-mendel non-coloc:  2327
total non-coloc:  2331
total mendelian:  4
total non-mendel:  2338
total     set:  2342

number of ecaviar_files:  3834
number of unique locus  126
colocalized mendelian:  0
colocalized non-mendel:  13
colocalized total:  13
non-colocalized mendelian:  4
non-mendel non-coloc:  2325
total non-coloc:  2329
total mendelian:  4
total non-mendel:  2338
total     set:  2342


### test for height

In [96]:
mendelian_new

['ENSG00000169604',
 'ENSG00000175054',
 'ENSG00000197299',
 'ENSG00000094804',
 'ENSG00000167513',
 'ENSG00000151849',
 'ENSG00000108821',
 'ENSG00000164692',
 'ENSG00000105664',
 'ENSG00000005339',
 'ENSG00000138346',
 'ENSG00000100393',
 'ENSG00000072840',
 'ENSG00000173040',
 'ENSG00000166147',
 'ENSG00000068078',
 'ENSG00000141756',
 'ENSG00000112964',
 'ENSG00000147099',
 'ENSG00000133703',
 'ENSG00000184634',
 'ENSG00000104320',
 'ENSG00000164190',
 'ENSG00000085840',
 'ENSG00000115947',
 'ENSG00000160299',
 'ENSG00000152952',
 'ENSG00000179295',
 'ENSG00000164754',
 'ENSG00000132155',
 'ENSG00000160957',
 'ENSG00000143622',
 'ENSG00000264229',
 'ENSG00000169071',
 'ENSG00000155850',
 'ENSG00000141646',
 'ENSG00000072501',
 'ENSG00000108055',
 'ENSG00000115904',
 'ENSG00000080603',
 'ENSG00000165392']

In [95]:
mendelian = list(mendelian_list[mendelian_list["Tissue"] == "Height"]["ENSG"])

mendelian_new = []

for ensg in mendelian:
    if ensg == None:
        continue        
    if "|" in ensg:
        mendelian_new.extend(ensg.split("|"))
    else:
        mendelian_new.append(ensg)

In [97]:
ecaviar_directory = '/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_output/Muscle_Skeletal/'

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.5)
print("")

get_ecaviar_results(mendelian_new, ecaviar_directory, 0.2)

number of ecaviar_files:  34605
number of unique locus  1459
colocalized mendelian:  0
colocalized non-mendel:  175
colocalized total:  175
non-colocalized mendelian:  25
non-mendel non-coloc:  11629
total non-coloc:  11654
total mendelian:  25
total non-mendel:  11804
total     set:  11829

number of ecaviar_files:  34605
number of unique locus  1459
colocalized mendelian:  0
colocalized non-mendel:  227
colocalized total:  227
non-colocalized mendelian:  25
non-mendel non-coloc:  11577
total non-coloc:  11602
total mendelian:  25
total non-mendel:  11804
total     set:  11829


In [32]:
#make output of LD files for eCAVIAR

ecaviar_directory = '/net/home/dlee/GTEx/gwas/zhang_2020_bc/eCAVIAR_results/eCAVIAR_output/Breast_Mammary_Tissue/'
# ecaviar_directory = '/net/data/GTEx/gwas/mahajan_2018_t2d/eCAVIAR_results/eCAVIAR_output/Whole_Blood/'
# ecaviar_directory = '/net/data/GTEx/gwas/ukbb_ldl_conditioned/eCAVIAR/eCAVIAR_output/Whole_Blood/'
# ecaviar_directory = '/net/data/GTEx/gwas/ukbb_height_conditioned/eCAVIAR/eCAVIAR_output/Muscle_Skeletal/'
# ecaviar_directory = '/net/data/GTEx/gwas/liu_2015_and_goyette_2015_ibd/cd_eCAVIAR/eCAVIAR_output/Colon_Sigmoid/'

efiles = glob.glob(ecaviar_directory + '*col')

print(len(efiles))

17846


In [37]:
efile_df

,Filename,Locus,Gene
11721,/net/home/dlee/GTEx/gwas/zhang_2020_bc/eCAVIAR...,305203,ENSG00000172554.7
8809,/net/home/dlee/GTEx/gwas/zhang_2020_bc/eCAVIAR...,305203,ENSG00000189292.11
14309,/net/home/dlee/GTEx/gwas/zhang_2020_bc/eCAVIAR...,305203,ENSG00000233296.1
8966,/net/home/dlee/GTEx/gwas/zhang_2020_bc/eCAVIAR...,305203,ENSG00000143727.11
14376,/net/home/dlee/GTEx/gwas/zhang_2020_bc/eCAVIAR...,305203,ENSG00000151353.10
...,...,...,...
16223,/net/home/dlee/GTEx/gwas/zhang_2020_bc/eCAVIAR...,243445116,ENSG00000179456.9
6110,/net/home/dlee/GTEx/gwas/zhang_2020_bc/eCAVIAR...,243445116,ENSG00000054282.11
15776,/net/home/dlee/GTEx/gwas/zhang_2020_bc/eCAVIAR...,243445116,ENSG00000224525.2
11091,/net/home/dlee/GTEx/gwas/zhang_2020_bc/eCAVIAR...,243445116,ENSG00000173728.6


## get all the CLPP for each locus (for chr 21)

In [3]:
import glob
import pandas as pd

In [4]:
#make output of LD files for eCAVIAR

ecaviar_directory = '/net/home/dlee/noah/eCAVIAR_output/'

efiles = glob.glob(ecaviar_directory + '*col')

In [20]:
position = [int(i.split("_")[2]) for i in efiles]
genes = [i.split("_")[3] for i in efiles]

In [45]:
data = {'Filename': efiles, 'Locus': position, 'Gene': genes}
efile_df = pd.DataFrame(data)

In [46]:
efile_df = efile_df.sort_values(['Locus'])


In [47]:
maxCLPP = []
maxPCausalset = []

for index, row in efile_df.iterrows():
    eCAVIAR_output = pd.read_csv(row["Filename"] , sep= '\t')
    maxCLPP.append(max(eCAVIAR_output["CLPP"]))
    maxPCausalset.append(max(eCAVIAR_output["Prob_in_pCausalSet"]))

In [51]:
efile_df["Maximum CLPP"] = maxCLPP
efile_df["Maximum Prob_in_pCausalSet"] = maxPCausalset

In [52]:
efile_df

,Filename,Locus,Gene,Maximum CLPP,Maximum Prop in Causal Set,Maximum Prob_in_pCausalSet
121,/net/home/dlee/noah/eCAVIAR_output/chr21_28442...,28442933,ENSG00000232692.1,0.000248,0.117522,0.117522
130,/net/home/dlee/noah/eCAVIAR_output/chr21_28442...,28442933,ENSG00000142192.16,0.000419,0.141913,0.141913
52,/net/home/dlee/noah/eCAVIAR_output/chr21_28442...,28442933,ENSG00000166265.7,0.000901,0.215824,0.215824
129,/net/home/dlee/noah/eCAVIAR_output/chr21_28442...,28442933,ENSG00000217026.3,0.000602,0.113724,0.113724
73,/net/home/dlee/noah/eCAVIAR_output/chr21_28442...,28442933,ENSG00000232079.2,0.000522,0.069609,0.069609
...,...,...,...,...,...,...
25,/net/home/dlee/noah/eCAVIAR_output/chr21_39604...,39604059,ENSG00000255568.2,0.004507,0.545419,0.545419
26,/net/home/dlee/noah/eCAVIAR_output/chr21_39604...,39604059,ENSG00000157538.9,0.000946,0.191796,0.191796
116,/net/home/dlee/noah/eCAVIAR_output/chr21_39604...,39604059,ENSG00000242553.1,0.001379,0.138242,0.138242
133,/net/home/dlee/noah/eCAVIAR_output/chr21_39604...,39604059,ENSG00000157557.7,0.000712,0.163720,0.163720


In [53]:
output_filename = 'eCAVIAR_chr21_summary.csv'
efile_df[["Locus", "Gene", "Maximum CLPP", "Maximum Prob_in_pCausalSet"]].to_csv(output_filename, index = False)